In [2]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 
device = "cuda"

In [3]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mne
from sklearn.preprocessing import StandardScaler

In [4]:
import import_ipynb

In [5]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        self.transforms = transforms.ToTensor()
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        x = self.x[index, ...]
        y = self.y[index, ...]
        return x, y
    
    def __len__(self):
        return len(self.x)

In [6]:
def Normalzie(data):
    normals = []
    scaler = StandardScaler()
    for idx in range(len(data)):
        normals.append(scaler.fit_transform(data[idx]))
    return np.array(normals)

In [7]:
PNES_data = mne.read_epochs(r"D:\MNE Data\PNES\Hosseinisamira.fif", preload=False).get_data(picks='eeg');
TLE_data = mne.read_epochs(r"D:\MNE Data\TLE\FarahmandianHakime.fif", preload=False).get_data(picks='eeg');

Reading D:\MNE Data\PNES\Hosseinisamira.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.67 ms
        0 CTF compensation matrices available
Not setting metadata
835 matching events found
No baseline correction applied
0 projection items activated
Loading data for 835 events and 3000 original time points ...
Reading D:\MNE Data\TLE\FarahmandianHakime.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.67 ms
        0 CTF compensation matrices available
Not setting metadata
399 matching events found
No baseline correction applied
0 projection items activated
Loading data for 399 events and 3000 original time points ...


In [8]:
PNES_data = Normalzie(PNES_data)
TLE_data = Normalzie(TLE_data)

In [9]:
PNES_labels = np.zeros((PNES_data.shape[0]))
TLE_labels = np.ones((TLE_data.shape[0]))

In [10]:
data = np.concatenate((PNES_data, TLE_data), axis=0).reshape(-1, 1, 19, 3000)
labels = np.concatenate((PNES_labels, TLE_labels))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [14]:
class network(nn.Module):
    def __init__(self, cnn_out_channels, cnn_kernel = 1, cnn_stride=1):
        super(network, self).__init__()
        self.cnn_out_channels = cnn_out_channels
        self.cnn_kernel = cnn_kernel
        self.cnn_stride = cnn_stride
        self.cnn = nn.Conv2d(1, self.cnn_out_channels, self.cnn_kernel, self.cnn_stride)
        self.lstm = nn.LSTM()

    def forward(self, x):
        return self.cnn(x)

In [15]:
b = torch.Tensor(479, 1, 19, 3000)
m = network(cnn_out_channels=1, cnn_kernel=19, cnn_stride=1)
m(b).shape

TypeError: __init__() missing 2 required positional arguments: 'input_size' and 'hidden_size'

In [ ]:
b = torch.Tensor(479, 1, 3000)
m = nn.LSTM(1000, 3001, batch_first= True)
m(b)[0].shape

NameError: name 'torch' is not defined